I was having this issue today where, when I went from a 10k dataset to a 30k dataset for the same type of data, my feed-forward neural net (NN) severely underperformed on the larger dataset.

**the NN is attempting to predict solvent density based on state variables, temperature and pressure, and a host of molecular features I feed in as inputs.**

You can see the NN's predictive failure in the image below where I've shown the predicted vs actual parity plots for the 10k (left) and 30k (right) datasets:

![](salty1.png)

At first I thought this must be a hyperparameter issue in the NN—a 10K dataset expanded to 30K with potentially different feature distributions could cause underperformance in the model training—so I did a quick check for this evaluating the performance of a shrinkage method, the Least Absolute Shrinkage Selection Operator or *LASSO*, on both datasets...

...and I found that LASSO was severely underperforming with the 30k dataset as well. So before attempting an exhaustive hyperparameter search I decided to check the underlying dataset.

Using another nifty statistical method, the bootstrap, I checked the confidence intervals of the features selected by LASSO. In the following image we're looking at a box and whiskers plot of the assigned coefficient value from LASSO for the indicated features for the 10k dataset (left) and 30k dataset (right):

![](salty2.png)

This gave me a couple of immediate concerns:

1. **Temperature** doesn't show up as an important state variable in the 30k dataset
2. The 30k dataset model has **really low coefficient values** compared to the 10k dataset

In other words:

1. *Based on thermodynamic principles, temperature and density are inversely correlated and this should certainly be identified by the LASSO*
2. *And if LASSO is selecting very low coefficients... it's suggesting none of the features are very correlated with density* 

## What's the deal?

Thermodynamics aside... if temperature is an important state variable in the subset model shouldn't it be included in the larger dataset model as well?

I took a look at the distribution of state variables for both datasets:

![](salty3.png)

Filtering the 30k dataset for the same distribution of state variables as the 10k dataset left me with about 28k data points and resulted in the same noisy parity plot as before:

![](salty4.png)

So I knew it wasn't an issue with disparate state variables. Expecting some more *fundamental* corruption of my data, I filtered both datasets down to a single datatype—a cation-anion pair—and checked the statistical summary of it's state variables:

![](salty5.png)

**viola!** There was stark disagreement between what should be *the same exact data* in both datasets:

<center> This was a dirty dataset problem. </center>

## Cleaning the dataset

The typical procedure here is to strategically sample from your data pipeline to find out where you've infected your dataset. I began at my local, unprocessed datafiles.

In the following surface plot for T-P-D of a single solvent, you can see sub-surfaces where a single experimental group produced many data points (these look like correlated gridpoints). So although there is noise in the data inter-experimentally, the intra-experimental data is acting as we expect it to: density decreases with temperature and increases with pressure. 

![](salty6.png)

Turns out there was a hiccup in my data processing immediately after this point, before adding molecular features to my dataframe, and upon cleanup:

![](salty7.png)

We get a nice regression!

The moral of the story here is: clean your data, carefully!